In [22]:
#Getting the data
import pandas as pd
from sets import Set

df = pd.read_csv("srep00196-s2.csv",header=None)
unique_ingredients = [i for i in df[0].unique()]
unique_ingredients = set(unique_ingredients + [i for i in df[1].unique()])

with open("Ingredients.csv","w") as f:
    for i in unique_ingredients:
        f.write(i + '\n')

In [ ]:
from scipy.sparse import coo_matrix,csr_matrix
from sklearn.neighbors import NearestNeighbors
from ingredientParser import ingredientParser as ingr_parser

class FlavorModel:
    
    def __init__(self,recipe_db,ingr_flavor_mapping,ingr_labels,flavor_labels):
        
        self.ingr_flavor_map = np.zeros((ingr_labels.shape[0],flavor_labels.shape[0]))

        self.ingr_map = dict()
        for row in ingr_labels.iterrows():
            self.ingr_map[row[1][1]] = row[1][0]

        for row in ingr_flavor_mapping.iterrows():
            self.ingr_flavor_map[row[1][0],row[1][1]] = 1
        
        self.parser = ingr_parser(self.ingr_map.keys())
        
        rsums = ingr_flavor_map.sum(axis = 1)
        self.ingr_flavor_map = ingr_flavor_map / rsums[:,np.newaxis]
        
        dishIndexList = []
        self.dishNameIndexMap = dict()
        self.dishIndexNameMap = dict()
        
        for index,dish in enumerate(receipe_db['title']):
            self.dishNameIndexMap[dish] = index
            self.dishIndexNameMap[index] = dish
            
            for ing in dish['ingredients']
                ingr_name = self.search_string(ing["text"])
                if ingr_name in self.ingr_map:
                    dishIndexList.append( (ing["weight"],index,self.ingr_map[ingr_name]) )
        
        data,row_index,col_index = zip(*dishIndexList)
        
        self.dish_matrix = coo_matrix( (data,(row_index,col_index)) ).tocsc()
        
        self.dish_mappings = np.dot(self.dish_matrix,self.ingr_flavor_map)
        
        self.nnModel = NearestNeighbors(n_neighbors=10,n_jobs=8)
        self.nnModel.fit(self.dish_mappings)
    
    def queryDish(self,dishName,numberNeighbors = 10):
        
        dishIndex = self.dishNameIndexMap[dishName]
        neighbors = self.nnModel.kneighbors(self.dish_matrix[dishIndex,:],n_neighbors = numberNeighbors)
        
        neighborNames = [self.dishIndexNameMap[i] for i in neighbors]
        return neighborNames
    
    def queryIngredients(self,ingrList,numberNeighbors = 10):
        
        example = np.zeros((,self.ingr_flavor_map.shape[0]))
        
        for ing in ingrList:
            if ingr_name in self.ingr_map:
                example[self.ingr_map[ingr_name]] += 1
        
        neighbors = self.nnModel.kneighbors(example,n_neighbors = numberNeighbors)
        
        neighborNames = [self.dishIndexNameMap[i] for i in neighbors]
        return neighborNames